https://medium.com/@aieeshashafique/feature-extraction-from-bert-25887ed2152a


In [ ]:
!pip install tensorflow
!pip install tensorflow_hub
!pip install bert-tensorflow
#!pip install bert-tensorflow==1.0.1
#!pip install bert-for-tf
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow.keras.models import Model 
import bert
import numpy as np
from numpy.linalg import norm


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 KB 4.1 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf

# Auxiliary components
import sys
sys.path.append("..")
#from nlp_utils import *

# BETO tokenizer
!pip install keras_bert
from keras_bert import load_vocabulary, Tokenizer
model_path = "drive/MyDrive/"
#https://github.com/dccuchile/beto/blob/master/config/uncased_2M/config.json
config_path = model_path + "config.json"
#https://github.com/guilopgar/ClinicalCodingTransformerES/tree/main/BETO/BETO-Galen
checkpoint_path = "drive/MyDrive/model.ckpt-1000000"
#https://github.com/dccuchile/beto/blob/master/config/uncased_2M/vocab.txt
vocab_file = "vocab.txt"
tokenizer = Tokenizer(token_dict=load_vocabulary(model_path + vocab_file), pad_index=1, cased=True)

# Hyper-parameters
training = False
trainable = True
SEQ_LEN = 128

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for keras_bert: filename=keras_bert-0.89.0-py3-none-any.whl size=33516 sha256=25b9508d1828e6e1d23e56c46f6714d6d879567bb56db3e61f4e8793ac9b2d69
  Stored in directory: /root/.cache/pip/wheels/a1/34/ed/6bbd71716d7bcea30d75e8bc5aeb94f4cb52636295c8343534
  Created wheel for keras-transformer: filename=keras_transformer-0.40.0-py3-none-any.whl size=12304 sha256=a58ac5930a2420b4679e83a95b9bfa9f85eb558e73b6642eb37d5ef7f071c62e
  Stored in directory: /root/.cache/pip/wheels/07/cd/a7/a8fa93f7e177eee0101fed63179f7a2fa3b53671ffaad82bfd
  Created wheel for keras-e

In [ ]:
from keras_bert import load_trained_model_from_checkpoint

model = load_trained_model_from_checkpoint(
    config_file=config_path, 
    checkpoint_file=checkpoint_path, 
    training=training,                                       
    trainable=trainable, 
    seq_len=SEQ_LEN
)

In [ ]:
# See BERT paper: https://arxiv.org/pdf/1810.04805.pdf
# And BERT implementation convert_single_example() at https://github.com/google-research/bert/blob/master/run_classifier.py

def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))


def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))


def get_ids(text, tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    #token_ids = tokenizer.convert_tokens_to_ids(tokens)
    token_ids = tokenizer.encode(text)[0]
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

In [ ]:
#import sys
#from absl import flags
#sys.argv=['preserve_unused_tokens=False']
#flags.FLAGS(sys.argv)

['preserve_unused_tokens=False']

In [ ]:
text = 'el paciente impaciente'
#marked_text = "[CLS] " + text + " [SEP]"
tokens = tokenizer.tokenize(text)
input_ids = get_ids(text, tokens, tokenizer, SEQ_LEN)
input_masks = get_masks(tokens,SEQ_LEN)
input_segments = get_segments(tokens,SEQ_LEN)
input_ids_tensor = tf.reshape(tf.convert_to_tensor(input_ids), [1, SEQ_LEN])
input_masks_tensor = tf.reshape(tf.convert_to_tensor(input_masks), [1, SEQ_LEN])
input_segments_tensor = tf.reshape(tf.convert_to_tensor(input_segments), [1, SEQ_LEN])
embeddings = model.predict([[input_ids_tensor],[input_segments_tensor]])
vector_A = np.mean(embeddings[0],axis=0)

1/1 [==============================] - 10s 10s/step


In [ ]:
text = 'el paciente impaciente'
#marked_text = "[CLS] " + text + " [SEP]"
tokens = tokenizer.tokenize(text)
input_ids = get_ids(text, tokens, tokenizer, SEQ_LEN)
input_masks = get_masks(tokens,SEQ_LEN)
input_segments = get_segments(tokens,SEQ_LEN)
input_ids_tensor = tf.reshape(tf.convert_to_tensor(input_ids), [1, SEQ_LEN])
input_masks_tensor = tf.reshape(tf.convert_to_tensor(input_masks), [1, SEQ_LEN])
input_segments_tensor = tf.reshape(tf.convert_to_tensor(input_segments), [1, SEQ_LEN])
embeddings = model.predict([[input_ids_tensor],[input_segments_tensor]])
vector_B = np.mean(embeddings[0],axis=0)

1/1 [==============================] - 1s 1s/step


In [ ]:

np.dot(vector_A,vector_B)/(norm(vector_A)*norm(vector_B))

1.0

* Prueba con un informe y la terminología clínica

In [ ]:
! python -m spacy download es
import spacy

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-10 15:16:03.646518: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
⚠ As of spaCy v3.0, shortcuts like 'es' are deprecated. Please use the
full pipeline package name 'es_core_news_sm' instead.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 13.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [ ]:
def NLP_preprocesaDocumento(doc):
  #Cargamos el modelo en español
  nlp = spacy.load("es_core_news_sm")

  #Tokenizado
  doc = [" ".join([token.text for token in nlp(doc)])][0]

  #Eliminación de signos de puntuación
  doc = [" ".join([token.lemma_ for token in nlp(doc) if not token.is_punct]) ][0]

  #Eliminación de stopwords
  spacy_stopwords = spacy.lang.es.stop_words.STOP_WORDS
  doc = [" ".join([token for token in doc.split() if not token.lower() in spacy_stopwords])][0]
  
  #Lematización de palabras
  doc = [" ".join([token.lemma_ for token in nlp(doc)])][0]

  #Eliminación de tildes
  tildes = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u', 'Á': 'A', 'E': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U', 'ü': 'u', 'Ü': 'U'}
  for tilde in tildes:
	  if tilde in doc:
		  doc = doc.replace(tilde, tildes[tilde])

  return doc


In [ ]:
import os
def cargaProblemasSaludAHSpainExtension():
  f = open('der2_Refset_ProblemasSaludAHSpainExtensionSnapshot_ES_20221201.txt')
  terminos_snomedct_problemassaludAH = []
  cont = 0
  for row in f:
    if cont>0:
      values = row.strip().split("\t")
      #id	effectiveTime	active	moduleId	refsetId	referencedComponentId	term
      #print(values)
      termino = {}
      termino["id"] = values[0];
      termino["effectiveTime"] = values[1];
      termino["active"] = values[2];
      termino["moduleId"] = values[3];
      termino["refsetId"] = values[4];
      termino["referencedComponentId"] = values[5];
      termino["term"] = values[6];
      termino["num_palabras"] = len(values[6].split(" "));
      #print(termino)
      terminos_snomedct_problemassaludAH.append(termino)
    cont = cont + 1

  f.close();
  return terminos_snomedct_problemassaludAH;

In [ ]:
terminos_snomedct_problemassaludAH = cargaProblemasSaludAHSpainExtension();

In [ ]:
def anadirEmbedding(model,lista,num_items_max=0,preprocesado=False):
  cont = 0
  for item in lista:
    #print(item["term"])
    if preprocesado:
      doc = NLP_preprocesaDocumento(item["term"])
      if doc == "":
        print("ERROR AL Parsear " + item["term"])
        doc = item["term"]
    else:
      doc = item["term"]
    #print(doc)

    tokens = tokenizer.tokenize(doc)
    input_ids = get_ids(doc, tokens, tokenizer, SEQ_LEN)
    input_masks = get_masks(tokens,SEQ_LEN)
    input_segments = get_segments(tokens,SEQ_LEN)
    input_ids_tensor = tf.reshape(tf.convert_to_tensor(input_ids), [1, SEQ_LEN])
    input_masks_tensor = tf.reshape(tf.convert_to_tensor(input_masks), [1, SEQ_LEN])
    input_segments_tensor = tf.reshape(tf.convert_to_tensor(input_segments), [1, SEQ_LEN])
    embeddings = model.predict([[input_ids_tensor],[input_segments_tensor]])
    item["embedding"] = np.mean(embeddings[0],axis=0)



    cont = cont + 1
    if cont == num_items_max:
      break
  return lista

In [ ]:
terminos_snomedct_problemassaludAH = anadirEmbedding(model, terminos_snomedct_problemassaludAH, 0, True);

1/1 [==============================] - 1s 521ms/step
ERROR AL Parsear estado de gran mal
1/1 [==============================] - 1s 521ms/step


In [ ]:
def NLP_get_ngrams(text, n):
    text = text.split()
    output = []
    for i in range(len(text) - n + 1):
        output.append(text[i:i+n])
    return output

In [ ]:
def getEmbedding(doc):
    tokens = tokenizer.tokenize(doc)
    input_ids = get_ids(doc, tokens, tokenizer, SEQ_LEN)
    input_masks = get_masks(tokens,SEQ_LEN)
    input_segments = get_segments(tokens,SEQ_LEN)
    input_ids_tensor = tf.reshape(tf.convert_to_tensor(input_ids), [1, SEQ_LEN])
    input_masks_tensor = tf.reshape(tf.convert_to_tensor(input_masks), [1, SEQ_LEN])
    input_segments_tensor = tf.reshape(tf.convert_to_tensor(input_segments), [1, SEQ_LEN])
    embeddings = model.predict([[input_ids_tensor],[input_segments_tensor]])
    return np.mean(embeddings[0],axis=0)


In [ ]:
def split_sentences(text):
    nlp = spacy.load("es_core_news_sm")
    tokens = nlp(text)
    sentences = []
    for sent in tokens.sents:
        sentences.append(sent.text.strip())
    return sentences

In [ ]:
informe = "El paciente presenta un aborto espontáneo y retenido, distendido, y es capaz de volverse inestable. El paciente presenta ansiedad y a veces un poco de afasia."
#informe = "El paciente presenta un abdomen distendido, y no es capaz de volverse loco. El paciente presenta ansiedad y a veces un poco de apraxia."
#informe = "Hay evidencia de hipoatenuación hepática difusa compatible con infiltración grasa . No hay dilatación de los conductos biliares intra o extrahepáticos. El paciente se encuentra en estado post colecistectomía. El bazo es normal. El páncreas es de contorno y características de atenuación normales. No hay evidencia de masa suprarrenal. Hay una hernia supraumbilical de tamaño moderado que contiene grasa. Los riñones son normales en tamaño, forma y configuración. No se identifican cálculos renales ni ureterales. No hay hidrouréter ni hidronefrosis. No hay evidencia de apendicitis. Hay varias asas de intestino delgado llenas de líquido, compatibles con una enteritis leve. No hay engrosamiento de la pared intestinal. No hay evidencia de obstrucción del intestino delgado o grueso. No hay evidencia de ascitis abdominal o linfadenopatía. No hay evidencia de masa vesical intrínseca o extrínseca . No hay ascitis pélvica ni linfadenopatía . El útero y los ovarios no presentan ninguna anomalía. Las imágenes de las bases pulmonares no muestran evidencia de masa pleural o parenquimatosa . No hay derrames pleurales. Hay cicatrices en el lóbulo medio derecho y en la língula, así como en ambas bases pulmonares. Las estructuras óseas están libres de lesiones líticas o blásticas. Se observan cambios degenerativos multinivel en la columna toracolumbar. Se observan calcificaciones dispersas en la aorta y en sus principales ramas, compatibles con la aterosclerosis."
informe_preprocesado = NLP_preprocesaDocumento(informe)

Opción 1 - Igual que fasttext

In [ ]:
import numpy as np
from numpy.linalg import norm
from operator import itemgetter

terminos_vinculados = []

#Empezamos por ngramas más grandes a más pequeños
for num_ngrams in range(6, 0, -1):
  ngramas = NLP_get_ngrams(informe_preprocesado, num_ngrams)
  for ngrama in ngramas:
    doc = " ".join([token for token in ngrama])
    #print(doc)
    doc_embedding = getEmbedding(doc)
    vector_A = doc_embedding
    #Por cada término de la terminología, comprobamos la similaridad con el ngrama previo
    terminos_candidatos = []
    for termino in terminos_snomedct_problemassaludAH:
      if "embedding" in termino:
        vector_B = termino["embedding"]
        cosine = np.dot(vector_A,vector_B)/(norm(vector_A)*norm(vector_B))
        if cosine >= 0.94:
          print(doc, " ", termino["term"], " Cosine Similarity:", cosine)
          terminos_candidatos.append({"term": termino["term"], "embedding": termino["embedding"], "cosine": cosine})
        #Nos quedamos con el término con mayor similaridad de los obtenidos
        if terminos_candidatos:
          terminos_candidatos_sorted = sorted(terminos_candidatos, key=itemgetter('cosine'), reverse=True) 
          terminos_vinculados.append(terminos_candidatos_sorted[0])

1/1 [==============================] - 1s 543ms/step
paciente presentar aborto espontaneo retenido distendido   abuso de cocaína sin adicción, en remisión  Cosine Similarity: 0.95210224
paciente presentar aborto espontaneo retenido distendido   distensión de manguito rotador de hombro  Cosine Similarity: 0.9418886
paciente presentar aborto espontaneo retenido distendido   hallazgo relacionado con el ronquido  Cosine Similarity: 0.94398725
paciente presentar aborto espontaneo retenido distendido   neoplasia de comportamiento incierto de glándula endocrina  Cosine Similarity: 0.9449975
paciente presentar aborto espontaneo retenido distendido   neoplasia maligna primaria de la unión rectosigmoidea  Cosine Similarity: 0.94849527
paciente presentar aborto espontaneo retenido distendido   ruptura de episiotomía en el puerperio  Cosine Similarity: 0.9487028
paciente presentar aborto espontaneo retenido distendido   várices esofágicas por cirrosis hepática  Cosine Similarity: 0.94830495
1/1 [=

In [ ]:
#Eliminamos duplicados
terminos_vinculados_sorted = sorted(terminos_vinculados, key=itemgetter('term')) 
termino_ant = {"term":"", "cosine": "0.0"}
terminos_vinculados_deduplicated = []
for termino in terminos_vinculados_sorted:
  if termino["term"] != termino_ant["term"]:
    if termino_ant["term"] != "":
      terminos_vinculados_deduplicated.append(termino_ant)
      print(termino_ant["term"], " Cosine Similarity:", termino_ant["cosine"])
    termino_ant = termino
  else:
    if termino_ant["cosine"]<termino["cosine"]:
      termino_ant = termino
terminos_vinculados_deduplicated.append(termino_ant)
print(termino_ant["term"], " Cosine Similarity:", termino_ant["cosine"])

abdomen distendido  Cosine Similarity: 0.95844823
aborto espontáneo  Cosine Similarity: 1.0
aborto espontáneo completo  Cosine Similarity: 0.9503943
aborto retenido  Cosine Similarity: 0.96006477
abuso de cocaína sin adicción, en remisión  Cosine Similarity: 0.95210224
accidente  Cosine Similarity: 0.9762807
amenaza de aborto  Cosine Similarity: 0.9767586
anemia hemolítica  Cosine Similarity: 0.942618
aneurisma intracraneal  Cosine Similarity: 0.95328516
ansiedad  Cosine Similarity: 0.99999994
cicatriz  Cosine Similarity: 0.98376924
cuerpo extraño en la vía respiratoria  Cosine Similarity: 0.94131345
dolor  Cosine Similarity: 0.9838431
efectos tardíos de la tuberculosis respiratoria  Cosine Similarity: 0.941278
fatiga  Cosine Similarity: 0.98586947
fractura de trocánter mayor  Cosine Similarity: 0.9457832
gota  Cosine Similarity: 0.9914494
hallazgo relacionado con el ronquido  Cosine Similarity: 0.9559584
hemorragia  Cosine Similarity: 0.9924431
neoplasia de comportamiento incierto de 

Opción 2 - Con frases completas

In [ ]:
#Troceamos en frases el contenido del informe
frases = split_sentences(informe)
terminos_vinculados = []
for frase in frases:
  #Empezamos por ngramas más grandes a más pequeños
  for num_ngrams in range(6, 0, -1):
    ngramas = NLP_get_ngrams(NLP_preprocesaDocumento(frase), num_ngrams)
    for ngrama in ngramas:
      doc = " ".join([token for token in ngrama])
      #print(doc)
      doc_embedding = getEmbedding(doc)
      vector_A = doc_embedding
      #Por cada término de la terminología, comprobamos la similaridad con el ngrama previo
      terminos_candidatos = []
      for termino in terminos_snomedct_problemassaludAH:
        if "embedding" in termino:
          vector_B = termino["embedding"]
          cosine = np.dot(vector_A,vector_B)/(norm(vector_A)*norm(vector_B))
          if cosine >= 0.94:
            print(doc, " ", termino["term"], " Cosine Similarity:", cosine)
            terminos_candidatos.append({"term": termino["term"], "embedding": termino["embedding"], "cosine": cosine})
          #Nos quedamos con el término con mayor similaridad de los obtenidos
          if terminos_candidatos:
            terminos_candidatos_sorted = sorted(terminos_candidatos, key=itemgetter('cosine'), reverse=True) 
            terminos_vinculados.append(terminos_candidatos_sorted[0])

1/1 [==============================] - 1s 527ms/step
paciente presentar aborto espontaneo retenido distendido   abuso de cocaína sin adicción, en remisión  Cosine Similarity: 0.95210224
paciente presentar aborto espontaneo retenido distendido   distensión de manguito rotador de hombro  Cosine Similarity: 0.9418886
paciente presentar aborto espontaneo retenido distendido   hallazgo relacionado con el ronquido  Cosine Similarity: 0.94398725
paciente presentar aborto espontaneo retenido distendido   neoplasia de comportamiento incierto de glándula endocrina  Cosine Similarity: 0.9449975
paciente presentar aborto espontaneo retenido distendido   neoplasia maligna primaria de la unión rectosigmoidea  Cosine Similarity: 0.94849527
paciente presentar aborto espontaneo retenido distendido   ruptura de episiotomía en el puerperio  Cosine Similarity: 0.9487028
paciente presentar aborto espontaneo retenido distendido   várices esofágicas por cirrosis hepática  Cosine Similarity: 0.94830495
1/1 [=

In [ ]:
#Eliminamos duplicados
terminos_vinculados_sorted = sorted(terminos_vinculados, key=itemgetter('term')) 
termino_ant = {"term":"", "cosine": "0.0"}
terminos_vinculados_deduplicated = []
for termino in terminos_vinculados_sorted:
  if termino["term"] != termino_ant["term"]:
    if termino_ant["term"] != "":
      terminos_vinculados_deduplicated.append(termino_ant)
      print(termino_ant["term"], " Cosine Similarity:", termino_ant["cosine"])
    termino_ant = termino
  else:
    if termino_ant["cosine"]<termino["cosine"]:
      termino_ant = termino
terminos_vinculados_deduplicated.append(termino_ant)
print(termino_ant["term"], " Cosine Similarity:", termino_ant["cosine"])

In [ ]:
frases